### 샘플 데이터 생성(광주은행 은행여신거래기본약관(가계용))

In [1]:
from pypdf import PdfReader
import re
import pandas as pd

reader = PdfReader("./sample_data/40-11-1001_은행여신거래기본약관(가계용)_20230502.pdf")
with open("./sample_data/거래여신기본약관.txt", 'w') as f:
    f.write(reader.pages[0].extract_text())
    f.write(reader.pages[1].extract_text())
    
with open("./sample_data/거래여신기본약관.txt",'r') as f:
    text = f.read()

lines = text.split("\n")

# 각 줄이 "제X조(" 또는 "①", "②", "③", "④", "⑤", "⑥", "⑦", "⑧"로 시작하는지 확인.
sections = []
current_section = []
for line in lines:
    if re.match("(제\\d+조\\()|(① )|(② )|(③ )|(④ )|(⑤ )|(⑥ )|(⑦ )|(⑧ )", line):
        # 새로운 섹션이 시작되면, 이전 섹션을 sections 리스트에 추가합니다.
        if current_section:
            sections.append("\n".join(current_section))
        current_section = [line]
    else:
        current_section.append(line)

# 마지막 섹션을 sections 리스트에 추가.
if current_section:
    sections.append("\n".join(current_section))
    
df = pd.DataFrame(dict(data=sections))

### 임베딩 모델 (sentence transformer 준비 및 milvus에 vector 데이터 삽입)

In [2]:
from langchain.vectorstores.milvus import Milvus
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(
    model_name = "jhgan/ko-sroberta-multitask"
)
embedding1 = HuggingFaceEmbeddings(
    model_name = "upskyy/kf-deberta-multitask",
)
# ds = load_dataset("maywell/ko_wikidata_QA", split="train")
# ds = ds.select(range(100))
# milvus data가 없을때 최초 생성용
# vec = Milvus.from_texts(
#     texts=df["data"].to_list(),
#     embedding=embedding,
#     collection_name="embedding",
# )
# vec2 = Milvus.from_texts(
#     texts=df["data"].to_list(),
#     embedding=embedding1,
#     collection_name="embedding1"
# )
vec = Milvus(
    embedding_function=embedding,
    collection_name="embedding",
)
vec1 = Milvus(
    embedding_function=embedding1,
    collection_name="embedding1",
)


### LLM 세팅

In [3]:
import sys
sys.path.append("/home/19kim/work/repos/singung")
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline, TextStreamer
from util import get_hf_model_tokenizer

model, tokenizer = get_hf_model_tokenizer(
    "/home/19kim/work/models/42dot_LLM-SFT-1.3B/",
)

pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    max_length=8192,
    streamer=TextStreamer(tokenizer=tokenizer, skip_prompt=True),
    repetition_penalty=1.1,
    top_k=10,
    top_p=0.97,
    temperature=0.1,
    do_sample=True,
    
)

llm = HuggingFacePipeline(
    pipeline=pipe,
)

`AnnotionFormat` is deprecated and will be removed in v4.38. Please use `transformers.image_utils.AnnotationFormat` instead.
The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### RAG 세팅

In [4]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnableParallel
from operator import itemgetter

SYSTEM_PROMPT = """### System:
당신은 사람들이 편하게 금융 상담을 할 수 있도록 도와주는 금융 상담 어시스턴트 입니다.
사람은 약관 검색 결과에 관한 정보에 도움을 얻고자 합니다.
### CONTEXT에 주어지는 정보를 바탕으로, 사람이 궁금해 하는 질문에 핵심적인 정보를 친절하고, 상세하게 설명하세요."""
template = """
### CONTEXT:
{context}
### USER:
사람: {question}
### ASSISTANT:
어시스턴트: """

template = SYSTEM_PROMPT + template
prompt = PromptTemplate.from_template(template)

memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question",
)
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
)
standalone_question = dict(
    standalone_question=dict(
        question=lambda x: x["question"],
        chat_history=lambda x: get_buffer_string(x["chat_history"]),
    )
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retriever = vec.as_retriever()

rag_chain = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

In [5]:
rag_chain_with_source = RunnableParallel(
    dict(
        context=retriever,
        question=RunnablePassthrough()
    )
).assign(answer=rag_chain)

In [17]:
rag_chain_with_source.invoke("채무자와 은행에 관한 내용 알려줘")


① 

/home/19kim/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


채무자는 은행이 채권보전상의 필요에 의하여 청구하는 때에는 부채현황, 채무자  
및 보증인의 신용상태나 담보의 상황에 관하여, 곧 회답하며, 또 은행이 그에 
관하여 조사하고자 할 때에는, 이에 협조하여야 합니다.
① 채무자가 발행․배서 등을 한 어음 또는 채무자가 은행에 제출한 각종 증서 
등이 불가항력 ․사변․재해․수송도중의 사고 등 은행 자신의 책임없는 사유로 인하여  
분실․손상․멸실 또는 늦게 도착한 경우 채무자는 은행의 장부ㆍ전표 등의 기록에  
의하여 채무를 갚기로 하되, 채무자가 은행의 장부ㆍ전표 등의 기록과 다른 자
자료를 제시할 경우 은행의 기록과 채무자가 제시하는 자료를 상호 대조하여 채무를 확정한 후 갚기로 합니다.② 채무자는 제1항의 분실ㆍ손상ㆍ멸실의 경우에 은행의 청구에 따라 곧 그에 
대신할 어음이나 증서 등을 제출하여야 합니다. 다만, 은행이 제3자와의 거래에서  
취득한 어음이나 증서의 경우에는 제출하지 않아도 됩니다.
③ 은행은 대출약정을 하기 전에 채무자가 미리 알 수 있도록 별도의 서면에 의<|endoftext|>


{'context': [Document(page_content='① 채무자는 은행이 채권보전상의 필요에 의하여 청구하는 때에는 부채현황 , 채무자  \n및 보증인의 신용상태나 담보의 상황에 관하여 , 곧 회답하며 , 또 은행이 그에 \n관하여 조사하고자 할 때에는 , 이에 협조하여야 합니다 .'),
  Document(page_content='① 채무자가 발행․배서 등을 한 어음 또는 채무자가 은행에 제출한 각종 증서 \n등이 불가항력 ․사변․재해․수송도중의 사고 등 은행 자신의 책임없는 사유로 인하여  \n분실․손상․멸실 또는 늦게 도착한 경우 채무자는 은행의 장부ㆍ전표 등의 기록에  \n의하여 채무를 갚기로 하되, 채무자가 은행의 장부ㆍ전표 등의 기록과 다른 자\n료를 제시할 경우 은행의 기록과 채무자가 제시하는 자료를 상호 대조하여 채\n무를 확정한 후 갚기로 합니다 .② 채무자는 제1항의 분실ㆍ손상ㆍ멸실의 경우에 은행의 청구에 따라 곧 그에 \n대신할 어음이나 증서 등을 제출하여야 합니다 . 다만, 은행이 제3자와의 거래에서  \n취득한 어음이나 증서의 경우에는 제출하지 않아도 됩니다 .'),
  Document(page_content='③ 은행은 대출약정을 하기 전에 채무자가 미리 알 수 있도록 별도의 서면에 의\n하여 약정이자 , 중도상환수수료 및 대출로 인하여 채무자가 부담하여야 할 부대\n비용의 항목과 금액을 설명하여야 합니다 . 아울러 약정이자와 그 명칭에 불구하고  \n채무자가 부담하는 이자와 성격이 유사한 수수료 등을 합산한 실질유효금리를 \n산정하여 설명하여야 합니다 .\n제4조의2(대출계약 철회)(이  조 항 의  밑 줄  친  부 분 은  공 정 거 래 위 원 회 의  표 준 약 관 과  다 릅 니 다 .)'),
  Document(page_content='① 이자ㆍ할인료ㆍ보증료ㆍ수수료 등(이하 “이자 등”이라고 합니다 )의 율ㆍ계산방법\nㆍ지급의 시기 및 방법에 관하여는 , 은행은 법령이 허용하는 한도 내에서 은행이 \n정할 수 있으

### 벡터디비 내용 삭제

In [6]:
from pymilvus import connections, db
con = connections.connect(host="127.0.0.1",port=19530)

# get milvus collection(something like column)
from pymilvus import Collection
col = Collection("embedding")

# drop collection
from pymilvus import utility
utility.drop_collection("embedding")